In [37]:
import numpy as np
import cv2 as cv
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [38]:
alex_data_dir = "./corner_detector_dataset/alex_data.csv"
javier_data_dir = "./corner_detector_dataset/javier_data.csv"
jose_data_dir = "./corner_detector_dataset/jose_data.csv"

alex_df = pd.read_csv(alex_data_dir)
javier_df = pd.read_csv(javier_data_dir)
jose_df = pd.read_csv(jose_data_dir)
full_df = pd.concat([alex_df, javier_df, jose_df])
images_dirs = full_df.image_dir.tolist()

In [39]:
image = cv.imread(images_dirs[0])
print(type(image))

<class 'numpy.ndarray'>


In [40]:
images = []
for image_name in images_dirs:
    image = cv.imread(image_name)
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image = np.reshape(image, (image.shape[0], image.shape[1], 1))
    image = image.tolist()
    images.append(image)
images = np.array(images)
print(images.shape)

(17442, 32, 32, 1)


In [41]:
full_df.head()

,image_dir,conv_a,conv_b,conv_c,conv_d,conv_e,conv_f,conv_g,conv_h,is_fridge_corner
0,./corner_detector_dataset/images/alex_16643254...,-1928.0,-2154.0,-2220.0,-2366.0,252.0,-252.0,186.0,-186.0,0.0
1,./corner_detector_dataset/images/alex_16643254...,-2089.0,-2313.0,-2357.0,-2587.0,271.0,-271.0,227.0,-227.0,0.0
2,./corner_detector_dataset/images/alex_16643254...,-2226.0,-2534.0,-2572.0,-2784.0,298.0,-298.0,260.0,-260.0,0.0
3,./corner_detector_dataset/images/alex_16643254...,-2469.0,-2737.0,-2723.0,-3065.0,291.0,-291.0,305.0,-305.0,0.0
4,./corner_detector_dataset/images/alex_16643254...,-2705.0,-2969.0,-2947.0,-3197.0,235.0,-235.0,257.0,-257.0,0.0


In [42]:
y_values = full_df.is_fridge_corner

In [43]:
test_ratio = 0.2
X_train, X_test, y_train, y_test = train_test_split(images, y_values, test_size=test_ratio)

In [48]:
# TODO: REMEMBER TO DO DATA NORMALIZATION BEFORE PASSING IT AS NN INPUT

size_of_kernel = (5,5)
pooling_downscale_factors = (2,2) # pooling layers will reduce the input to half width and half height

model = keras.models.Sequential()
model.add( keras.layers.Conv2D(20, size_of_kernel, input_shape = (32, 32, 1), activation='relu') )
model.add( keras.layers.Dropout(0.4) )
model.add( keras.layers.Conv2D(20, size_of_kernel, activation='relu') )
model.add( keras.layers.Dropout(0.4) )
model.add( keras.layers.AveragePooling2D(pooling_downscale_factors) )
model.add( keras.layers.Conv2D(20, size_of_kernel, activation='relu') )
model.add( keras.layers.Dropout(0.4) )
model.add( keras.layers.Conv2D(20, size_of_kernel, activation='relu') )
model.add( keras.layers.Dropout(0.4) )
model.add( keras.layers.Flatten() )
model.add( keras.layers.Dense(320, activation='relu') )
model.add( keras.layers.Dropout(0.4) )
model.add( keras.layers.Dense(160, activation='relu') )
model.add( keras.layers.Dropout(0.4) )
model.add( keras.layers.Dense(20, activation='relu') )
model.add( keras.layers.Dropout(0.4) )
model.add( keras.layers.Dense(1, activation='sigmoid') )
#model.add(keras.layers.Flatten(input_shape=[28, 28]))

In [49]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_39 (Conv2D)          (None, 28, 28, 20)        520       
                                                                 
 dropout_31 (Dropout)        (None, 28, 28, 20)        0         
                                                                 
 conv2d_40 (Conv2D)          (None, 24, 24, 20)        10020     
                                                                 
 dropout_32 (Dropout)        (None, 24, 24, 20)        0         
                                                                 
 average_pooling2d_9 (Averag  (None, 12, 12, 20)       0         
 ePooling2D)                                                     
                                                                 
 conv2d_41 (Conv2D)          (None, 8, 8, 20)          10020     
                                                     

In [50]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999),                                                                                                                                                                       
              loss=tf.keras.losses.BinaryCrossentropy(),                                                                                                                                                                         
              metrics=[tf.keras.metrics.Accuracy()]) 

In [51]:

history = model.fit(X_train, y_train, epochs=30, validation_split=0.1)

Epoch 1/30
393/393 [==============================] - 18s 45ms/step - loss: 0.1689 - accuracy: 0.0125 - val_loss: 0.0662 - val_accuracy: 0.0000e+00
Epoch 2/30
393/393 [==============================] - 18s 46ms/step - loss: 0.0789 - accuracy: 0.0000e+00 - val_loss: 0.0515 - val_accuracy: 0.0000e+00
Epoch 3/30
393/393 [==============================] - 19s 47ms/step - loss: 0.0707 - accuracy: 0.0000e+00 - val_loss: 0.0577 - val_accuracy: 0.0000e+00
Epoch 4/30
393/393 [==============================] - 19s 48ms/step - loss: 0.0698 - accuracy: 7.1673e-04 - val_loss: 0.0484 - val_accuracy: 0.0000e+00
Epoch 5/30
198/393 [==============>...............] - ETA: 9s - loss: 0.0759 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()